In [ ]:
# imports
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# debugging
import threading
import time

# cur_time = 0
# def increment_time(var):
#     var += 1
# timer = threading.Timer(0.100, increment_time(cur_time))

# def start_checkpoint(counter):
#     tmr = threading.Timer(0.100, increment_time(counter))
#     tmr.start()
#     return tmr

# def stop_checkpoint(counter, tmr):
#     tmr.cancel()
#     return cur_time - counter

In [ ]:
# setup
# fig, ax = plt.subplots()
# plt.axis('off')
# grid = ax.table(cellText=grid_data, loc='center')

In [ ]:
# Controller Class
class Controller:

    def __init__(self, gamemode, seed, animations):
        self.grid = None
        self.gamemode = gamemode
        self.seed = seed
        self.animations = animations
    


In [ ]:
# The Grid class
# Reason for creating the Grid class is so that I can use "id" as key in dictionary rather than converting the coord to string
class Grid:

    # TODO: add args and kwards so that custom grid can be passed
    def __init__(self, x_size, y_size, low, high):
        self.size = x_size * y_size
        self.x_size = x_size
        self.y_size = y_size
        self.grid = np.random.randint(low+1, high=high, size=(1, self.size))[0]

    # returns the right cell
    def get_right_cell(self, id):
        lid = id+1
        if id < self.size and (lid) % self.x_size != 0:
            return (lid, self.grid[lid])

    # returns the bottom cell
    def get_bottom_cell(self, id):
        lid = id+self.x_size
        if lid < self.size:
            return (lid, self.grid[lid])

    # returns the cell
    def get_cell(self, id):
        return self.grid[id]

    # converts id to coordinates (x,y)
    def id_to_coord(self, id):
        lid = id+1
        return (lid // self.x_size, lid % self.y_size)

    # pretty prints grid
    def spit_grid(self):
        for k, c in enumerate(self.grid, start=1):
            print(c, end=' | ')
            if k % self.x_size == 0:
                print('\n --- ')

In [ ]:
# Algorithm Super Class
class PFindAlgo:
    
    def __init__(self, grid):
        if grid.size == 0:
            raise Exception('Cannot initialize a grid with size 0')
        self.grid = grid

In [ ]:
# Heuristic Algorithm Class
class SafatAlgo(PFindAlgo):

    def __init__(self, grid):
        super().__init__(grid)
        self.memory = {0: (grid[0], None)}  # {point: [weight_from_origin, previous_point]}
        self.queue = [0]


    # Help find the algorithm
    def find_shortest_path(self):

        # Step
        while len(self.queue) > 0:

            # get next point in queue
            vis = self.q_get()
            vis_point = self.mem_get_point(vis)

            # add right side to memory and queue
            right = grid.get_right_cell(vis)
            if right:
                self.mem_add_point(right, vis, vis_point)
                self.q_enqueue(right[0])

            # add left side to memory and queue
            bottom = grid.get_bottom_cell(vis)
            if bottom:
                self.mem_add_point(bottom, vis, vis_point)
                self.q_enqueue(bottom[0])


    # Backtrack -> returns path in reverse
    def backtrack_path(self):

        # init target info
        target_cell = grid.size-1
        path = [target_cell]

        ptr = self.mem_get_point(target_cell)

        # start from target [bottom right] backtrack via 'prev' until [0,0]
        while ptr[1]:
            path.append(ptr[1])
            ptr = self.mem_get_point(ptr[1])
        path.append(0)
        return path
    

    # Helper functions
    def q_enqueue(self, c):
        self.queue.append(c)

    def q_get(self):
        next = self.queue[0]
        self.queue.pop(0)
        return next

    def mem_get_point(self, k):
        try:
            return self.memory[k]
        except KeyError:
            return None

    def mem_add_point(self, cell, prev_id, prev):
        wfo = prev[0]+cell[1]
        point = self.mem_get_point(cell[0])

        # if a point is not in memory or the wfo is less than current path -> add/replace point
        if point is None or wfo < point[0]:
            self.memory[cell[0]] = (wfo, prev_id)


In [ ]:
# Dijkstra Algorithm Class
class DijkstraAlgo:
    
    def __init__(self, grid):
        super().__init__(grid)
        


In [ ]:
controller = Controller(0, 1, False)
grid = Grid(15,15,0,5)
grid.spit_grid()

In [ ]:
algo = SafatAlgo(grid.grid)
algo.find_shortest_path()

print(algo.memory)
print(algo.backtrack_path())